# Import some packages

In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import shutil
import time
from copy import deepcopy

device = 'cuda'
import torchvision.models as models

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

# Load data

In [3]:
train_df = pd.read_csv("../data/kaggle/train.csv")
test_df = pd.read_csv("../data/kaggle/test.csv")
print(f"Train dataframe has shape: {train_df.shape}")
print(f"Test dataframe has shape: {test_df.shape}")
display(train_df.head())
display(test_df.head())

Train dataframe has shape: (31390, 5)
Test dataframe has shape: (2413, 4)


,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


# Transforming 
This will convert each protein sequence (a list of amino acids) to a matrix, where each row is the physical and chemical properties of the corresponding amino acid

In [4]:
df_aa = pd.read_csv("../data/aminoacids.csv")
transform_dict = {}
"""
0 = Name (Ignore)
1 = Abbr (Ignore)
2 = Letter
3 = Molecular Weight
4 = Molecular Formula
5 = Residue Formula (Ignore)
6 = Residue Weight (Ignore)
7 = pKa1
8 = pKb2
9 = pKx3
10 = pl4
11 = H
12 = VSC
13 = P1
14 = P2
15 = SASA
16 = NCISC
17 = carbon
18 = hydrogen
19 = nitrogen
20 = oxygen
21 = sulfur
"""
feature_list = [3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
df_aa = df_aa.fillna(method="bfill")
display(df_aa)
df_aa.iloc[:, feature_list] = (df_aa.iloc[:, feature_list] - df_aa.iloc[:, feature_list].mean()) / df_aa.iloc[:, feature_list].std()
display(df_aa)

,Name,Abbr,Letter,Molecular Weight,Molecular Formula,Residue Formula,Residue Weight,pKa1,pKb2,pKx3,...,VSC,P1,P2,SASA,NCISC,carbon,hydrogen,nitrogen,oxygen,sulfur
0,Alanine,Ala,A,89.10,C3H7NO2,C3H5NO,71.08,2.34,9.69,8.18,...,27.5,8.1,0.046,1.181,0.007187,3,7,1,2,0
1,Cysteine,Cys,C,121.16,C3H7NO2S,C3H5NOS,103.15,1.96,10.28,8.18,...,44.6,5.5,0.128,1.461,-0.036610,3,7,1,2,1
2,Aspartic acid,Asp,D,133.11,C4H7NO4,C4H5NO3,115.09,1.88,9.60,3.65,...,40.0,13.0,0.105,1.587,-0.023820,4,7,1,4,0
3,Glutamic acid,Glu,E,147.13,C5H9NO4,C5H7NO3,129.12,2.19,9.67,4.25,...,62.0,12.3,0.151,1.862,0.006802,5,9,1,4,0
4,Phenylalanine,Phe,F,165.19,C9H11NO2,C9H9NO,147.18,1.83,9.13,6.00,...,115.5,5.2,0.290,2.228,0.037552,9,11,1,2,0
5,Glycine,Gly,G,75.07,C2H5NO2,C2H3NO,57.05,2.34,9.60,6.00,...,0.0,9.0,0.000,0.881,0.179052,2,5,1,2,0
6,Histidine,His,H,155.16,C6H9N3O2,C6H7N3O,137.14,1.82,9.17,6.00,...,79.0,10.4,0.230,2.025,-0.010690,6,9,3,2,0
7,Isoleucine,Ile,I,131.18,C6H13NO2,C6H11NO,113.16,2.36,9.60,10.53,...,93.5,5.2,0.186,1.810,0.021631,6,13,1,2,0
8,Lysine,Lys,K,146.19,C6H14N2O2,C6H12N2O,128.18,2.18,8.95,10.53,...,100.0,11.3,0.219,2.258,0.017708,6,14,2,2,0
9,Leucine,Leu,L,131.18,C6H13NO2,C6H11NO,113.16,2.36,9.60,12.48,...,93.5,4.9,0.186,1.931,0.051672,6,13,1,2,0


,Name,Abbr,Letter,Molecular Weight,Molecular Formula,Residue Formula,Residue Weight,pKa1,pKb2,pKx3,...,VSC,P1,P2,SASA,NCISC,carbon,hydrogen,nitrogen,oxygen,sulfur
0,Alanine,Ala,A,-1.621141,C3H7NO2,C3H5NO,-1.621221,0.730019,0.583220,-0.500359,...,-1.238768,-0.038297,-1.347287,-1.374663,-0.514757,-1.091228,-1.058615,-0.513697,-0.743392,-0.308957
1,Cysteine,Cys,C,-0.530193,C3H7NO2S,C3H5NOS,-0.530003,-0.876023,1.967169,-0.500359,...,-0.752971,-1.034028,-0.471113,-0.747549,-1.094269,-1.091228,-1.058615,-0.513697,-0.743392,3.089572
2,Aspartic acid,Asp,D,-0.123554,C4H7NO4,C4H5NO3,-0.123731,-1.214137,0.372109,-2.101702,...,-0.883653,1.838272,-0.716870,-0.465347,-0.925034,-0.620502,-1.058615,-0.513697,2.230176,-0.308957
3,Glutamic acid,Glu,E,0.353524,C5H9NO4,C5H7NO3,0.353655,0.096055,0.536307,-1.889604,...,-0.258652,1.570190,-0.225357,0.150568,-0.519851,-0.149776,-0.269139,-0.513697,2.230176,-0.308957
4,Phenylalanine,Phe,F,0.968076,C9H11NO2,C9H9NO,0.968167,-1.425458,-0.730358,-1.270983,...,1.261237,-1.148920,1.259864,0.970296,-0.112974,1.733127,0.520336,-0.513697,-0.743392,-0.308957
5,Glycine,Gly,G,-2.098559,C2H5NO2,C2H3NO,-2.098607,0.730019,0.372109,-1.270983,...,-2.020019,0.306379,-1.838800,-2.046571,1.759321,-1.561954,-1.848090,-0.513697,-0.743392,-0.308957
6,Histidine,His,H,0.626771,C6H9N3O2,C6H7N3O,0.626544,-1.467722,-0.636531,-1.270983,...,0.224303,0.842541,0.618761,0.515638,-0.751301,0.320949,-0.269139,1.997710,-0.743392,-0.308957
7,Isoleucine,Ile,I,-0.189229,C6H13NO2,C6H11NO,-0.189402,0.814547,0.372109,0.330359,...,0.636236,-1.148920,0.148619,0.034104,-0.323637,0.320949,1.309812,-0.513697,-0.743392,-0.308957
8,Lysine,Lys,K,0.321537,C6H14N2O2,C6H12N2O,0.321670,0.053791,-1.152580,0.330359,...,0.820895,1.187217,0.501226,1.037487,-0.375545,0.320949,1.704549,0.742007,-0.743392,-0.308957
9,Leucine,Leu,L,-0.189229,C6H13NO2,C6H11NO,-0.189402,0.814547,0.372109,1.019679,...,0.636236,-1.263812,0.148619,0.305107,0.073859,0.320949,1.309812,-0.513697,-0.743392,-0.308957


In [5]:
# feature_list chooses which physical and chemical properties to inlcude
def getTransformDict(feature_list = [3, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]):
  transform_dict = {}
  for index, row in df_aa.iterrows():
    row_values = row.values
    letter = row_values[2]
    the_rest = [row_values[i] for i in feature_list]
    transform_dict.update({letter: np.array(the_rest, dtype="float")})
  transform_dict.update({None: np.zeros(shape=len(feature_list),dtype="float")})
  return transform_dict

In [6]:
# convert list of amino acids into a matrix of their corresponding physical and chemical properties
def convertSequences(sequences, MAX_LENGTH, transform_dict=None):
  if transform_dict is None:
    transform_dict = getTransformDict()
  sequenceList = []
  for sequence in sequences:
    letterMatrix = []
    for i in range(MAX_LENGTH):
      if i < len(sequence):
        letterMatrix.append(transform_dict[sequence[i]])
      else:
        letterMatrix.append(transform_dict[None])
    sequenceList.append(np.array(letterMatrix))
  return sequenceList

In [7]:
def getSequences(df, MAX_LENGTH=224):
  df.reset_index(inplace=True)
  sequences = [list(string) for string in df["protein_sequence"].values.tolist()]
  return convertSequences(sequences, MAX_LENGTH)

In [8]:
from sklearn.model_selection import train_test_split

train_df["protein_sequence_len"] = train_df["protein_sequence"].apply(lambda x: len(x))
MAX_LENGTH = 224
MIN_LENGTH = 100
train_df = train_df[train_df["protein_sequence_len"] <= MAX_LENGTH]
train_df = train_df[train_df["protein_sequence_len"] >= MIN_LENGTH]
X = getSequences(train_df, MAX_LENGTH=MAX_LENGTH)
y = train_df['tm'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train))

6201


In [9]:
test_df["protein_sequence_len"] = test_df["protein_sequence"].apply(lambda x: len(x))
test_df = test_df[test_df["protein_sequence_len"] <= MAX_LENGTH]
test_df = test_df[test_df["protein_sequence_len"] >= MIN_LENGTH]
X_test = getSequences(test_df)

## Loading and normalizing

In [10]:
def toTensor(array):
    tensor = torch.Tensor(array)
    return tensor.view(tensor.size(dim=0), 1, tensor.size(dim=1), tensor.size(dim=2))

In [11]:
from torch.utils.data import TensorDataset, DataLoader
trainset = TensorDataset(toTensor(X_train), torch.Tensor(y_train))
valset = TensorDataset(toTensor(X_val), torch.Tensor(y_val))
testset = TensorDataset(toTensor(X_test), torch.Tensor(np.zeros(shape=len(X_test))))
print(trainset)

#TODO: revise batch_size
BATCH_SIZE = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

KeyboardInterrupt: 

# Define the model

In [ ]:
class HoltryNet(nn.Module):
    def __init__(self, num_classes=1):
        super(HoltryNet, self).__init__()
        self.lstm = nn.LSTM(
            input_size=(len(feature_list)),
            hidden_size=256,                  # try 128, 256
            num_layers=1,
            batch_first=True
        )

        self.fc = nn.Linear(len(feature_list))
    def forward(self, x):
        
        return out

In [ ]:
model = Net().to(device)
# model = HoltryNet().to(device)

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# Define the Training Functions

In [ ]:
%%time
best_val_loss = 100000
best_val_model = None
MAX_EPOCH = 30
loss_record = {'train': [], 'dev': []} # added
for epoch in range(MAX_EPOCH):  
    model.train()
    running_train_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.view(outputs.size(dim=0))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_train_loss += loss.item()
        out = outputs.detach()
        assert out.shape == labels.shape
        
    loss_record['train'].append(running_train_loss/len(trainset))
    
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs,labels in valloader:
            out = model(inputs.cuda()).cpu()
            out = out.detach().view(out.size(dim=0))
            running_val_loss += (torch.pow(out - labels, 2).sum().item()) ** 0.5
    loss_record['dev'].append(running_val_loss/len(trainset))
    print("Epoch [{:>2} / {}]: Train loss = {:<20} Val Loss = {:<20}".format(epoch+1, MAX_EPOCH, running_train_loss/len(trainset), running_val_loss/len(valset)))
    if running_val_loss < best_val_loss:
        best_val_loss = running_val_loss
        best_val_model = deepcopy(model.state_dict())
    lr_scheduler.step()
    
print('Finished Training')  

In [ ]:
# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

In [ ]:
def plot_learning_curve(record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    x_1 = range(len(record['train']))
    x_2 = range(len(record['dev']))
    figure(figsize=(6, 4))
    plt.plot(x_1, record['train'], c='tab:red', label='train')
    plt.plot(x_2, record['dev'], c='tab:cyan', label='dev')
    
    # TODO: feel free to change this range to see the learning curve better
    y_min = min(min(record['train']), min(record['dev']))
    y_max = max(max(record['train']), max(record['dev']))
    plt.ylim(max(.95*y_min-0.01, 0), 1.05*y_max+0.01)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()

In [ ]:
plot_learning_curve(loss_record, title="CNN model")

In [ ]:
%%time
preds = []
model.load_state_dict(best_val_model)
model.eval()
with torch.no_grad():
    for inputs,labels in testloader:
        out = model(inputs.cuda()).cpu()
        out = out.view(out.size(dim = 0))
        preds.append(out.detach().cpu())
preds = torch.cat(preds, dim=0).numpy()

In [ ]:
import csv

def save_prediction(predictions, startAt = 0):
  with open('pred.csv', 'w') as fp:
          writer = csv.writer(fp)
          writer.writerow(['seq_id', 'tm'])
          for i, p in enumerate(predictions):
              writer.writerow([i+startAt, p])

save_prediction(preds, startAt = 31390)